In [144]:
import osmnx as ox 
ox.config(use_cache=True, log_console=True)
import math

In [131]:
place = "Uniondale, NY, United States"
graph = ox.graph_from_place(place, network_type='drive')
nodes, streets = ox.graph_to_gdfs(graph)

C:\Users\dylan\anaconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [155]:
class SearchNode:
    def __init__(self, id = 0, cost = 0, action = "", parent = None, depth = 0):
        self.cost = cost
        self.action = action
        self.parent = parent
        self.id = id
        self.depth = depth
        
    def __gt__(self, other):
        return (self.cost > other.cost)
        
    
    def print_route(self):
        current = self
        while current.parent != None:
            outputstr = str(current.depth) + "\t Street: " + str(current.action) + "\t " + str(current.cost) + " meters"
            print(outputstr)
            current = current.parent

In [177]:
def uniform_cost_search(graph, start, goal):
    
    start_node = SearchNode(start) #initialize the start node
    queue = [(0, start_node)] #create the fringe priority queue
    visited = [] #initialize the visited list
    nodes_visited = 0 #variable to keep track of the number of nodes visited
    
    while(len(queue)>0):#continue while fringe is not empty or is 
        queue.sort()
        (cost, cur_node) = queue.pop(0) #pop the lowest value
        nodes_visited+=1
        if(cur_node.id == goal):
            #return solution (final search node)
            print("Nodes visited: " + str(nodes_visited))
            return cur_node
        visited.append(cur_node.id) #add the current node to the visited array
        for child in graph[cur_node.id]: #iterate over children nodes
            
            #create child search node object
            child_node = SearchNode(child) #initiate the child node
            child_node.cost = cur_node.cost + graph[cur_node.id][child][0]['length']#calculate the new cost
            child_node.parent = cur_node #save parent
            child_node.depth = cur_node.depth + 1 #increase depth
            if 'name' in graph[cur_node.id][child][0]: #assugn the action to either the street name (if present) or osmid
                child_node.action = graph[cur_node.id][child][0]['name']#save street name as action
            else:
                child_node.action = graph[cur_node.id][child][0]['osmid']#save street id if no name is present
            
            #check if the child has been visited (if it has, disregard)
            if child not in visited:
                
                in_queue = False #boolean to check wheter or not the child node is already in the fringe queue
                
                for i in range(0, len(queue)):#iterate over fringe to check whether child is already there
                    if(queue[i][1].id == child_node.id):#child is in the fringe queue
                        if(queue[i][0] > child_node.cost):#a shorter path is found
                            new_entry = (child_node.cost, child_node)
                            queue[i] = new_entry #replace the entry in the fringe
                            in_queue = True #signify that child has been found in fringe
                
                if(not in_queue):#if its not already in the queue, add it
                    queue.append((child_node.cost, child_node))
        
    return None

In [183]:
start_node_id = 261345717
end_node_id = 261345849
print("Performing UCS algorithm...")
print("Starting point node with id: " + str(start_node_id) + " with coordinates (x, y): (" + str(nodes.loc[start_node_id]['x']) + ', ' + str(nodes.loc[start_node_id]['y']) + ')')
print("Ending point node with id: " + str(start_node_id) + " with coordinates (x, y): (" + str(nodes.loc[end_node_id]['x']) + ', ' + str(nodes.loc[end_node_id]['y']) + ')')
print()
end_node = uniform_cost_search(graph, start_node_id, end_node_id)
if end_node != None:
    print("Total cost (in meters): " + str(end_node.cost))
    print()
    end_node.print_route()
else:
    print("No path found")

Performing UCS algorithm...
Starting point node with id: 261345717 with coordinates (x, y): (-73.591487, 40.725556)
Ending point node with id: 261345717 with coordinates (x, y): (-73.607599, 40.687838)

Nodes visited: 20963
Total cost (in meters): 5350.770999999998

45	 Street: Milburn Avenue	 5350.770999999998 meters
44	 Street: Milburn Avenue	 4936.195999999998 meters
43	 Street: Harold Avenue	 4733.906999999998 meters
42	 Street: Harold Avenue	 4416.125999999998 meters
41	 Street: Argyle Avenue	 4323.791999999999 meters
40	 Street: Argyle Avenue	 4197.522999999998 meters
39	 Street: Argyle Avenue	 4060.5649999999987 meters
38	 Street: Argyle Avenue	 3923.647999999999 meters
37	 Street: ['Clinton Avenue', 'Argyle Avenue']	 3786.5519999999988 meters
36	 Street: Newton Avenue	 3645.4309999999987 meters
35	 Street: Newton Avenue	 3563.618999999999 meters
34	 Street: Newton Avenue	 3379.083999999999 meters
33	 Street: Uniondale Avenue	 3287.2169999999987 meters
32	 Street: Uniondale Aven

In [151]:
def heuristic_function(start_node, end_node):
    xdelta = start_node['x'] - end_node['x']
    ydelta = start_node['y'] - end_node['y'] 
    distance = math.sqrt(xdelta**2 + ydelta**2) * 87843.36 #pythagorean theorem to find the diagonal distance times the length of one degree
    return distance

In [175]:
def astar_search(graph, start, goal):
    
    start_node = SearchNode(start) #initialize the start node
    queue = [(0 + heuristic_function(nodes.loc[start], nodes.loc[goal]), start_node)] #create the fringe priority queue
    visited = [] #initialize the visited list
    nodes_visited = 0 #variable to keep track of the number of nodes visited
    
    while(len(queue)>0):#continue while fringe is not empty 
        queue.sort()
        (cost, cur_node) = queue.pop(0) #pop the lowest value
        nodes_visited+=1
        if(cur_node.id == goal):
            #return solution (final search node)
            print("Nodes visited: " + str(nodes_visited))
            return cur_node
        visited.append(cur_node.id) #add the current node to the visited array
        for child in graph[cur_node.id]: #iterate over children nodes
            
            #create child search node object
            child_node = SearchNode(child) #initiate the child node
            
            child_node.cost = cur_node.cost + graph[cur_node.id][child][0]['length'] #calculate the new cost
            child_node.parent = cur_node #save parent
            child_node.depth = cur_node.depth + 1 #increase depth
            if 'name' in graph[cur_node.id][child][0]: #assugn the action to either the street name (if present) or osmid
                child_node.action = graph[cur_node.id][child][0]['name']#save street name as action
            else:
                child_node.action = graph[cur_node.id][child][0]['osmid']#save street id if no name is present
            
            #check if the child has been visited (if it has, disregard)
            if child not in visited:
                
                in_queue = False #boolean to check wheter or not the child node is already in the fringe queue
                approximated_cost = heuristic_function(nodes.loc[child], nodes.loc[goal])
                for i in range(0, len(queue)):#iterate over fringe to check whether child is already there
                    if(queue[i][1].id == child_node.id):#child is in the fringe queue
                        if(queue[i][0] > child_node.cost + approximated_cost):#a shorter path is found
                            new_entry = (child_node.cost + approximated_cost, child_node)
                            queue[i] = new_entry #replace the entry in the fringe
                            in_queue = True #signify that child has been found in fringe
                
                if(not in_queue):#if its not already in the queue, add it
                    queue.append((child_node.cost + approximated_cost, child_node))
        
    return None

In [181]:
start_node_id = 261345717
end_node_id = 261345849
print("Performing A* algorithm...")
print("Starting point node with id: " + str(start_node_id) + " with coordinates (x, y): (" + str(nodes.loc[start_node_id]['x']) + ', ' + str(nodes.loc[start_node_id]['y']) + ')')
print("Ending point node with id: " + str(start_node_id) + " with coordinates (x, y): (" + str(nodes.loc[end_node_id]['x']) + ', ' + str(nodes.loc[end_node_id]['y']) + ')')
print()
end_node = astar_search(graph, start_node_id, end_node_id)
if end_node != None:
    print("Total cost (in meters): " + str(end_node.cost))
    print()
    end_node.print_route()
else:
    print("No path found")

Performing A* algorithm...
Starting point node with id: 261345717 with coordinates (x, y): (-73.591487, 40.725556)
Ending point node with id: 261345717 with coordinates (x, y): (-73.607599, 40.687838)

Nodes visited: 6177
Total cost (in meters): 5350.770999999998

45	 Street: Milburn Avenue	 5350.770999999998 meters
44	 Street: Milburn Avenue	 4936.195999999998 meters
43	 Street: Harold Avenue	 4733.906999999998 meters
42	 Street: Harold Avenue	 4416.125999999998 meters
41	 Street: Argyle Avenue	 4323.791999999999 meters
40	 Street: Argyle Avenue	 4197.522999999998 meters
39	 Street: Argyle Avenue	 4060.5649999999987 meters
38	 Street: Argyle Avenue	 3923.647999999999 meters
37	 Street: ['Clinton Avenue', 'Argyle Avenue']	 3786.5519999999988 meters
36	 Street: Newton Avenue	 3645.4309999999987 meters
35	 Street: Newton Avenue	 3563.618999999999 meters
34	 Street: Newton Avenue	 3379.083999999999 meters
33	 Street: Uniondale Avenue	 3287.2169999999987 meters
32	 Street: Uniondale Avenue